In [73]:
import backtrader as bt
from backtrader.feeds import GenericCSVData
from nsepy import get_history
from datetime import date
import pandas as pd

#Download the Data and Convert to CSV format in Pandas Dataframe

try:
    data = pd.read_csv('TQQQ.csv')
except:
    data = get_history(symbol="TQQQ", start=date(2020,1,1), end=date(2021,2,16))
    data.to_csv('TQQQ.csv')

In [74]:
data.head()

,Date,High,Low,Open,Close,Volume,Adj Close
0,2010-02-11,0.869792,0.811146,0.813750,0.865104,1728000.0,0.862942
1,2010-02-12,0.876146,0.836667,0.841563,0.868646,4300800.0,0.866475
2,2010-02-16,0.904375,0.875104,0.889063,0.902292,4809600.0,0.900037
3,2010-02-17,0.917813,0.900625,0.914375,0.917604,9590400.0,0.915311
4,2010-02-18,0.940521,0.911146,0.916667,0.935000,19430400.0,0.932663


In [80]:
btdata = GenericCSVData(dataname='TQQQ.csv',
                     dtformat=('%Y-%m-%d'),
                     datetime=0,
                     high=5,
                     low=6,
                     open=4,
                     close=8,
                     volume=10,
                     openinterest=-1,
                     #fromdate=date(2021, 2, 16),
                     #todate=date(2021, 2, 16)
                    )

In [82]:
# Create a Trading Strategy

class SmaCross(bt.Strategy):
    alias = ('SMA_CrossOver',)
    
    # list of parameters which are configurable for the strategy
    params = dict(
        pfast=10,  # period for the fast moving average
        pslow=30   # period for the slow moving average
    )
    
    def log(self, txt, dt=None):
        ''' Logging function for this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))  

    def __init__(self):
        sma1 = bt.ind.SMA(period=self.p.pfast)  # fast moving average
        sma2 = bt.ind.SMA(period=self.p.pslow)  # slow moving average
        self.crossover = bt.ind.CrossOver(sma1, sma2)  # crossover signal
        self.log(f'Initial portfolio value of {self.broker.get_value():.2f}\n')
        

    def next(self):
        if not self.position:  # not in the market
            if self.crossover > 0:  # if fast crosses slow to the upside
                self.buy()  # enter long
                self.log(f'BUY {self.getsizing()} shares of {self.data._name} at {self.data.close[0]}')

        elif self.crossover < 0:  # in the market & cross to the downside
            self.sell() #exit long
            self.log(f'CLOSE LONG position of {self.position.size} shares '
                         f'of {self.data._name} at {self.data.close[0]:.2f}')